In [1]:
import torch
from torch import nn
import pdb
import os
from transformers import RobertaTokenizer, RobertaModel, RobertaForMaskedLM, RobertaConfig

In [12]:
# Make sure we're in the transformers directory with fine-tuned model output.
os.chdir('/home/jupyter/Notebooks/crystal/NLP/transformers/examples/')
os.getcwd()

'/home/jupyter/Notebooks/crystal/NLP/transformers/examples'

In [13]:
def make_vocab(vocab_file):
    vocab = []
    with open(vocab_file, 'r') as v:
        vocab = v.read().splitlines()
    return vocab

In [6]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_wiki-103/')

model = RobertaForMaskedLM.from_pretrained('./output_wiki-103/')

config = RobertaConfig.from_pretrained('./output_wiki-103/')
config.output_hidden_states = True

In [7]:
# Adapted from transformer docs at https://huggingface.co/transformers/model_doc/roberta.html?highlight=vocabulary#transformers.RobertaTokenizer.save_vocabulary
# and https://github.com/huggingface/transformers/issues/2072
# See https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_roberta.py
# for the word_embeddings definition and related functions.
# More helpful info on output of the forward pass and hidden layers is here:
# https://github.com/huggingface/transformers/issues/1827

# input_ids = torch.tensor(tokenizer.encode("disgusted", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
# print(input_ids[0][1].item())

test_word = torch.tensor([tokenizer.encode("disgusted")])
# Print the index of the test word.
print(test_word[0][1].item())

32759


In [8]:
print(type(model))
print(model)

<class 'transformers.modeling_roberta.RobertaForMaskedLM'>
RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [9]:
# Try getting both input and output embeddings using transformers methods.
output_embeddings_test = model.get_output_embeddings()
print(output_embeddings_test,'\n')
input_embeddings_test = model.get_input_embeddings()
print(input_embeddings_test,'\n------------------------')

# Print the value of the input embeddings for our test word.
# These values match what is extracted manually in RoBERTa_testing-1.py
print(input_embeddings_test(torch.LongTensor([32759])))

Linear(in_features=768, out_features=50265, bias=True) 

Embedding(50265, 768, padding_idx=1) 
------------------------
tensor([[ 5.2666e-02, -1.7146e-01,  1.6461e-02, -3.7467e-02, -1.3350e-01,
          1.1051e-01, -1.2718e-01, -8.1737e-02,  3.5000e-02, -3.2664e-02,
         -1.2303e-02,  2.1744e-01, -7.6540e-02,  2.6898e-02, -5.9862e-03,
         -1.1723e-01,  9.6832e-03, -1.8487e-01,  4.3507e-02,  1.9149e-02,
         -4.4449e-02, -4.5171e-02,  2.4407e-02, -1.6115e-01,  3.1173e-02,
         -4.0548e-02,  3.5446e-02,  1.3479e-02, -2.2354e-01, -6.2794e-02,
         -2.5782e-02,  4.1355e-02,  5.2338e-02,  9.5993e-02, -1.3193e-01,
          4.9301e-02, -4.5019e-02,  3.0012e-02, -5.5757e-02, -2.4916e-01,
          6.3038e-02, -5.4601e-02, -1.2683e-01,  3.0680e-01,  7.3173e-02,
         -5.6098e-02, -1.7094e-02, -1.2768e-01,  1.5285e-01, -1.0663e-01,
          1.5613e-02,  1.1909e-01, -1.0941e-01, -6.1781e-03, -6.4058e-02,
          1.6774e-01,  6.7063e-02,  2.0279e-02, -2.2349e-01, -5.38

In [103]:
# Try getting various embeddings as output by the RoBERTa model.
# These values are different from the input values because the model
# is returning a contextual embedding for the single test word given.
embedding_test = model.roberta.embeddings(test_word)
embedding_test_word = model.roberta.embeddings.word_embeddings(test_word)
print(embedding_test[0][0][0],'\n\n')
print(embedding_test_word[0][0][0].item())

tensor(0.1768, grad_fn=<SelectBackward>) 


0.15049785375595093


In [60]:
# Try extracting embeddings using hidden states directly.
output = model(test_word)
print(f'There are {len(output)} outputs.')
hidden_states = output[-1]
print(f'There are {len(hidden_states)} hidden states.')
embedding_test_hidden = hidden_states[0]
print(embedding_test_hidden)

There are 2 outputs.
There are 13 hidden states.
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.0982, -0.3758,  0.1873,  ..., -0.2205, -0.0786,  0.2975],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)


In [62]:
# Or try using the sequence output at the last layer of the model.
# This probably won't be useful for my task. Output values match
# the prediction scores
output = model(test_word)
sequence_output = output[0]
print(sequence_output)

tensor([[[26.1772, -5.4167, 13.9335,  ...,  0.9339,  2.4213, 10.8156],
         [ 0.2701, -5.7334,  5.5102,  ..., -4.2464, -3.5145,  1.1312],
         [ 8.6342, -6.2831, 12.5975,  ..., -2.8398, -1.4640,  5.4114]]],
       grad_fn=<AddBackward0>)


In [72]:
input_ids = torch.tensor(tokenizer.encode("disgusted", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores)

tensor(1.1502, grad_fn=<NllLossBackward>)
tensor([[[26.1772, -5.4167, 13.9335,  ...,  0.9339,  2.4213, 10.8156],
         [ 0.2701, -5.7334,  5.5102,  ..., -4.2464, -3.5145,  1.1312],
         [ 8.6342, -6.2831, 12.5975,  ..., -2.8398, -1.4640,  5.4114]]],
       grad_fn=<AddBackward0>)


In [14]:
vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
vocab = make_vocab(vocab_file)

In [112]:
################################################
# This cell will write out input embeddings    #
# for all the words in my vocabulary, using    #
# pre-trained RoBERTa with no fine-tuning.     #
################################################
# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

model = RobertaForMaskedLM.from_pretrained('roberta-base', config=config)

config = RobertaConfig.from_pretrained('roberta-base')
config.output_hidden_states = True
input_embeddings = model.get_input_embeddings()
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_input.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
#     print(input_embeddings_test(torch.LongTensor([v_tensor[0][1].item()])))
    v_embed = input_embeddings(torch.LongTensor([v_tensor[0][1].item()]))
#     for n in range(v_embed.size()[1])
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
Saved the embedding for aback.
abashed is at index 4091
Saved the embedding for abashed.
abhor is at index 35350
Saved the embedding for abhor.
abhorred is at index 35350
Saved the embedding for abhorred.
abhorrence is at index 35350
Saved the embedding for abhorrence.
abhorrent is at index 35350
Saved the embedding for abhorrent.
abominable is at index 4091
Saved the embedding for abominable.
abound is at index 32937
Saved the embedding for abound.
absent is at index 11640
Saved the embedding for absent.
absorbed is at index 22416
Saved the embedding for absorbed.
acceptance is at index 10502
Saved the embedding for acceptance.
accepted is at index 3903
Saved the embedding for accepted.
accepting is at index 8394
Saved the embedding for accepting.
accommodating is at index 33681
Saved the embedding for accommodating.
accomplished is at index 9370
Saved the embedding for accomplished.
accordant is at index 10170
Saved the embedding for accordant.
accursed is at 

Saved the embedding for befuddled.
begging is at index 22901
Saved the embedding for begging.
begrudge is at index 28
Saved the embedding for begrudge.
begrudging is at index 28
Saved the embedding for begrudging.
begrudgingly is at index 28
Saved the embedding for begrudgingly.
beguiled is at index 21422
Saved the embedding for beguiled.
belated is at index 12138
Saved the embedding for belated.
belittling is at index 12138
Saved the embedding for belittling.
belligerence is at index 35756
Saved the embedding for belligerence.
belligerent is at index 35756
Saved the embedding for belligerent.
belonging is at index 11441
Saved the embedding for belonging.
bemused is at index 28
Saved the embedding for bemused.
bemusement is at index 28
Saved the embedding for bemusement.
benevolence is at index 42364
Saved the embedding for benevolence.
benevolent is at index 43186
Saved the embedding for benevolent.
benumbed is at index 21576
Saved the embedding for benumbed.
berate is at index 14719


Saved the embedding for clutched.
cluttered is at index 29409
Saved the embedding for cluttered.
cockeyed is at index 740
Saved the embedding for cockeyed.
cockiness is at index 24231
Saved the embedding for cockiness.
cocksure is at index 740
Saved the embedding for cocksure.
cocky is at index 24231
Saved the embedding for cocky.
cognizant is at index 28105
Saved the embedding for cognizant.
cold is at index 2569
Saved the embedding for cold.
collected is at index 4786
Saved the embedding for collected.
collusive is at index 9843
Saved the embedding for collusive.
colonized is at index 17735
Saved the embedding for colonized.
combative is at index 14960
Saved the embedding for combative.
comedic is at index 29045
Saved the embedding for comedic.
comfort is at index 5863
Saved the embedding for comfort.
comfortable is at index 3473
Saved the embedding for comfortable.
comforted is at index 5863
Saved the embedding for comforted.
comical is at index 3137
Saved the embedding for comical.

Saved the embedding for demented.
demised is at index 4410
Saved the embedding for demised.
demoralized is at index 36810
Saved the embedding for demoralized.
demure is at index 4410
Saved the embedding for demure.
denied is at index 2296
Saved the embedding for denied.
denouncing is at index 32439
Saved the embedding for denouncing.
depleted is at index 26391
Saved the embedding for depleted.
deplorable is at index 28156
Saved the embedding for deplorable.
deprecating is at index 8273
Saved the embedding for deprecating.
depressed is at index 16658
Saved the embedding for depressed.
depression is at index 6943
Saved the embedding for depression.
deprived is at index 22632
Saved the embedding for deprived.
deranged is at index 1935
Saved the embedding for deranged.
derision is at index 1935
Saved the embedding for derision.
derisive is at index 1935
Saved the embedding for derisive.
derogatory is at index 30971
Saved the embedding for derogatory.
desire is at index 4724
Saved the embed

Saved the embedding for distraught.
distress is at index 13250
Saved the embedding for distress.
distressed is at index 21460
Saved the embedding for distressed.
distressing is at index 7018
Saved the embedding for distressing.
distrust is at index 27948
Saved the embedding for distrust.
distrustful is at index 27948
Saved the embedding for distrustful.
distrusting is at index 27948
Saved the embedding for distrusting.
disturbed is at index 22938
Saved the embedding for disturbed.
diverted is at index 19070
Saved the embedding for diverted.
dodgy is at index 25744
Saved the embedding for dodgy.
doleful is at index 109
Saved the embedding for doleful.
doltish is at index 385
Saved the embedding for doltish.
dominant is at index 7353
Saved the embedding for dominant.
dominating is at index 17349
Saved the embedding for dominating.
domineering is at index 13567
Saved the embedding for domineering.
done is at index 626
Saved the embedding for done.
doomed is at index 23326
Saved the embedd

...

Saved the embedding for piqued.
pissed is at index 34449
Saved the embedding for pissed.
pitiable is at index 8516
Saved the embedding for pitiable.
pitiful is at index 8516
Saved the embedding for pitiful.
pity is at index 31373
Saved the embedding for pity.
pitying is at index 31373
Saved the embedding for pitying.
placated is at index 15155
Saved the embedding for placated.
placation is at index 15155
Saved the embedding for placation.
placid is at index 15155
Saved the embedding for placid.
plain is at index 10798
Saved the embedding for plain.
plaintive is at index 46560
Saved the embedding for plaintive.
planning is at index 1884
Saved the embedding for planning.
playful is at index 23317
Saved the embedding for playful.
playfully is at index 310
Saved the embedding for playfully.
pleading is at index 17532
Saved the embedding for pleading.
pleasant is at index 16219
Saved the embedding for pleasant.
pleased is at index 4343
Saved the embedding for pleased.
pleasing is at index 2

Saved the embedding for resting.
restless is at index 36844
Saved the embedding for restless.
restlessness is at index 1079
Saved the embedding for restlessness.
restrained is at index 25063
Saved the embedding for restrained.
restraint is at index 20219
Saved the embedding for restraint.
retaliating is at index 18570
Saved the embedding for retaliating.
retaliatory is at index 18570
Saved the embedding for retaliatory.
rethinking is at index 769
Saved the embedding for rethinking.
reticence is at index 5494
Saved the embedding for reticence.
reticent is at index 5494
Saved the embedding for reticent.
revengeful is at index 13543
Saved the embedding for revengeful.
reverent is at index 26911
Saved the embedding for reverent.
revolted is at index 34633
Saved the embedding for revolted.
revulsion is at index 6910
Saved the embedding for revulsion.
righteous is at index 37909
Saved the embedding for righteous.
rigid is at index 24577
Saved the embedding for rigid.
riled is at index 910
Sa

snickering is at index 4543
Saved the embedding for snickering.
snide is at index 4543
Saved the embedding for snide.
sniggering is at index 4543
Saved the embedding for sniggering.
sniveling is at index 4543
Saved the embedding for sniveling.
snobbish is at index 4543
Saved the embedding for snobbish.
snobby is at index 4543
Saved the embedding for snobby.
snooty is at index 4543
Saved the embedding for snooty.
snotty is at index 579
Saved the embedding for snotty.
sociable is at index 17380
Saved the embedding for sociable.
soft is at index 3793
Saved the embedding for soft.
solemn is at index 29807
Saved the embedding for solemn.
solicitous is at index 22706
Saved the embedding for solicitous.
solitary is at index 24429
Saved the embedding for solitary.
solitude is at index 41813
Saved the embedding for solitude.
somber is at index 16487
Saved the embedding for somber.
somberly is at index 16487
Saved the embedding for somberly.
somnolent is at index 16487
Saved the embedding for so

Saved the embedding for ticked.
tickled is at index 10457
Saved the embedding for tickled.
tied is at index 3016
Saved the embedding for tied.
tiered is at index 3318
Saved the embedding for tiered.
tight is at index 3229
Saved the embedding for tight.
tightlipped is at index 3229
Saved the embedding for tightlipped.
timid is at index 39649
Saved the embedding for timid.
timidly is at index 39649
Saved the embedding for timidly.
timidness is at index 39649
Saved the embedding for timidness.
tired is at index 7428
Saved the embedding for tired.
tiredly is at index 7428
Saved the embedding for tiredly.
tiredness is at index 7428
Saved the embedding for tiredness.
titillated is at index 13515
Saved the embedding for titillated.
tolerant is at index 32836
Saved the embedding for tolerant.
tongue is at index 15686
Saved the embedding for tongue.
tormented is at index 16535
Saved the embedding for tormented.
touched is at index 6699
Saved the embedding for touched.
tough is at index 1828
Sav

Saved the embedding for uppity.
upset is at index 4904
Saved the embedding for upset.
uptight is at index 18256
Saved the embedding for uptight.
useless is at index 23584
Saved the embedding for useless.
vacant is at index 11042
Saved the embedding for vacant.
vacuous is at index 18721
Saved the embedding for vacuous.
vanquished is at index 44400
Saved the embedding for vanquished.
vehement is at index 45373
Saved the embedding for vehement.
vengeful is at index 748
Saved the embedding for vengeful.
venomous is at index 32051
Saved the embedding for venomous.
vex is at index 37894
Saved the embedding for vex.
vexation is at index 37894
Saved the embedding for vexation.
vexed is at index 37894
Saved the embedding for vexed.
vicious is at index 16339
Saved the embedding for vicious.
victorious is at index 22518
Saved the embedding for victorious.
vigilant is at index 17258
Saved the embedding for vigilant.
vile is at index 32359
Saved the embedding for vile.
villainous is at index 17031


In [95]:
################################################
# This cell will write out input embeddings    #
# for all the words in my vocabulary, using    #
# RoBERTa fine-tuned on wiki-103 training text.#
################################################
# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_wiki-103/')

model = RobertaForMaskedLM.from_pretrained('./output_wiki-103/', config=config)

config = RobertaConfig.from_pretrained('./output_wiki-103/')
config.output_hidden_states = True
input_embeddings = model.get_input_embeddings()
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_input_wiki-103.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
#     print(input_embeddings_test(torch.LongTensor([v_tensor[0][1].item()])))
    v_embed = input_embeddings(torch.LongTensor([v_tensor[0][1].item()]))
#     for n in range(v_embed.size()[1])
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
Saved the embedding for aback.
abashed is at index 4091
Saved the embedding for abashed.
abhor is at index 35350
Saved the embedding for abhor.
abhorred is at index 35350
Saved the embedding for abhorred.
abhorrence is at index 35350
Saved the embedding for abhorrence.
abhorrent is at index 35350
Saved the embedding for abhorrent.
abominable is at index 4091
Saved the embedding for abominable.
abound is at index 32937
Saved the embedding for abound.
absent is at index 11640
Saved the embedding for absent.
absorbed is at index 22416
Saved the embedding for absorbed.
acceptance is at index 10502
Saved the embedding for acceptance.
accepted is at index 3903
Saved the embedding for accepted.
accepting is at index 8394
Saved the embedding for accepting.
accommodating is at index 33681
Saved the embedding for accommodating.
accomplished is at index 9370
Saved the embedding for accomplished.
accordant is at index 10170
Saved the embedding for accordant.
accursed is at 

Saved the embedding for belligerence.
belligerent is at index 35756
Saved the embedding for belligerent.
belonging is at index 11441
Saved the embedding for belonging.
bemused is at index 28
Saved the embedding for bemused.
bemusement is at index 28
Saved the embedding for bemusement.
benevolence is at index 42364
Saved the embedding for benevolence.
benevolent is at index 43186
Saved the embedding for benevolent.
benumbed is at index 21576
Saved the embedding for benumbed.
berate is at index 14719
Saved the embedding for berate.
berating is at index 14719
Saved the embedding for berating.
bereaved is at index 17738
Saved the embedding for bereaved.
bereft is at index 17738
Saved the embedding for bereft.
beseeching is at index 9988
Saved the embedding for beseeching.
bested is at index 275
Saved the embedding for bested.
betrayal is at index 26760
Saved the embedding for betrayal.
betrayed is at index 26913
Saved the embedding for betrayed.
bewildered is at index 33304
Saved the embed

Saved the embedding for comical.
commanding is at index 20510
Saved the embedding for commanding.
commiserating is at index 7034
Saved the embedding for commiserating.
commiserative is at index 7034
Saved the embedding for commiserative.
communicative is at index 16759
Saved the embedding for communicative.
compassion is at index 14736
Saved the embedding for compassion.
compassionate is at index 23303
Saved the embedding for compassionate.
competent is at index 17451
Saved the embedding for competent.
competitive is at index 2695
Saved the embedding for competitive.
complacence is at index 13000
Saved the embedding for complacence.
complacency is at index 13000
Saved the embedding for complacency.
complacent is at index 13000
Saved the embedding for complacent.
complacently is at index 13000
Saved the embedding for complacently.
complain is at index 11316
Saved the embedding for complain.
complaining is at index 13689
Saved the embedding for complaining.
composed is at index 14092
Sav

despair is at index 21508
Saved the embedding for despair.
despaired is at index 2694
Saved the embedding for despaired.
despairing is at index 21508
Saved the embedding for despairing.
desperate is at index 7764
Saved the embedding for desperate.
desperation is at index 24278
Saved the embedding for desperation.
despise is at index 43255
Saved the embedding for despise.
despondent is at index 18690
Saved the embedding for despondent.
destitute is at index 15357
Saved the embedding for destitute.
destroyed is at index 4957
Saved the embedding for destroyed.
detached is at index 27687
Saved the embedding for detached.
determination is at index 8964
Saved the embedding for determination.
determined is at index 3030
Saved the embedding for determined.
determining is at index 13684
Saved the embedding for determining.
deterred is at index 10922
Saved the embedding for deterred.
detest is at index 6769
Saved the embedding for detest.
detestable is at index 6769
Saved the embedding for detes

Saved the embedding for dubious.
dubiously is at index 30180
Saved the embedding for dubiously.
dull is at index 22018
Saved the embedding for dull.
dumb is at index 16881
Saved the embedding for dumb.
dumbfound is at index 16881
Saved the embedding for dumbfound.
dumbfounded is at index 16881
Saved the embedding for dumbfounded.
dumbstruck is at index 16881
Saved the embedding for dumbstruck.
dumfounded is at index 385
Saved the embedding for dumfounded.
dupe is at index 4279
Saved the embedding for dupe.
duplicitous is at index 30501
Saved the embedding for duplicitous.
dysphoric is at index 44153
Saved the embedding for dysphoric.
eager is at index 7921
Saved the embedding for eager.
eagerness is at index 7921
Saved the embedding for eagerness.
earnest is at index 22623
Saved the embedding for earnest.
easy is at index 1365
Saved the embedding for easy.
ebullient is at index 364
Saved the embedding for ebullient.
ecstasy is at index 37695
Saved the embedding for ecstasy.
ecstatic is

...

nausea is at index 27214
Saved the embedding for nausea.
nauseated is at index 39117
Saved the embedding for nauseated.
nauseous is at index 39117
Saved the embedding for nauseous.
needy is at index 28166
Saved the embedding for needy.
nefarious is at index 33952
Saved the embedding for nefarious.
negating is at index 15183
Saved the embedding for negating.
negative is at index 2430
Saved the embedding for negative.
negativity is at index 30269
Saved the embedding for negativity.
neglected is at index 20428
Saved the embedding for neglected.
nerdy is at index 38286
Saved the embedding for nerdy.
nerved is at index 295
Saved the embedding for nerved.
nerves is at index 17358
Saved the embedding for nerves.
nervous is at index 7464
Saved the embedding for nervous.
nervously is at index 40968
Saved the embedding for nervously.
nervousness is at index 7464
Saved the embedding for nervousness.
nescient is at index 295
Saved the embedding for nescient.
nettled is at index 1161
Saved the embe

pranking is at index 3349
Saved the embedding for pranking.
precarious is at index 27180
Saved the embedding for precarious.
predatory is at index 29216
Saved the embedding for predatory.
prejudiced is at index 34286
Saved the embedding for prejudiced.
preoccupied is at index 1198
Saved the embedding for preoccupied.
prepared is at index 2460
Saved the embedding for prepared.
preparing is at index 4568
Saved the embedding for preparing.
pretending is at index 23748
Saved the embedding for pretending.
pretentious is at index 11857
Saved the embedding for pretentious.
prideful is at index 7040
Saved the embedding for prideful.
priggish is at index 3349
Saved the embedding for priggish.
primed is at index 32575
Saved the embedding for primed.
private is at index 940
Saved the embedding for private.
processing is at index 5774
Saved the embedding for processing.
propositioning is at index 16104
Saved the embedding for propositioning.
proud is at index 2602
Saved the embedding for proud.
pr

Saved the embedding for satisfy.
saturnine is at index 4005
Saved the embedding for saturnine.
saucy is at index 2241
Saved the embedding for saucy.
savage is at index 32264
Saved the embedding for savage.
scandalized is at index 4220
Saved the embedding for scandalized.
scare is at index 13207
Saved the embedding for scare.
scared is at index 8265
Saved the embedding for scared.
scary is at index 10222
Saved the embedding for scary.
scattered is at index 12827
Saved the embedding for scattered.
schadenfreude is at index 8447
Saved the embedding for schadenfreude.
scheming is at index 30315
Saved the embedding for scheming.
scoffer is at index 34564
Saved the embedding for scoffer.
scoffing is at index 34564
Saved the embedding for scoffing.
scorn is at index 38430
Saved the embedding for scorn.
scorned is at index 2850
Saved the embedding for scorned.
scornful is at index 38430
Saved the embedding for scornful.
scowl is at index 2850
Saved the embedding for scowl.
scowling is at index

stifling is at index 1690
Saved the embedding for stifling.
still is at index 202
Saved the embedding for still.
stillness is at index 202
Saved the embedding for stillness.
stimulated is at index 42040
Saved the embedding for stimulated.
stinky is at index 1690
Saved the embedding for stinky.
stirred is at index 26158
Saved the embedding for stirred.
stoic is at index 20572
Saved the embedding for stoic.
stoical is at index 20572
Saved the embedding for stoical.
stolid is at index 1690
Saved the embedding for stolid.
stoned is at index 1690
Saved the embedding for stoned.
storming is at index 2130
Saved the embedding for storming.
stormy is at index 2130
Saved the embedding for stormy.
stout is at index 34636
Saved the embedding for stout.
straight is at index 1359
Saved the embedding for straight.
strained is at index 15718
Saved the embedding for strained.
strange is at index 7782
Saved the embedding for strange.
stressed is at index 5882
Saved the embedding for stressed.
stricken i

Saved the embedding for uncommitted.
uncommunicative is at index 32275
Saved the embedding for uncommunicative.
uncomprehending is at index 32275
Saved the embedding for uncomprehending.
uncompromising is at index 32213
Saved the embedding for uncompromising.
unconcerned is at index 28198
Saved the embedding for unconcerned.
unconfident is at index 542
Saved the embedding for unconfident.
unconvinced is at index 28198
Saved the embedding for unconvinced.
uncooperative is at index 542
Saved the embedding for uncooperative.
uncurious is at index 16511
Saved the embedding for uncurious.
undecided is at index 28598
Saved the embedding for undecided.
underhanded is at index 223
Saved the embedding for underhanded.
understanding is at index 2969
Saved the embedding for understanding.
undesirable is at index 39028
Saved the embedding for undesirable.
unease is at index 12515
Saved the embedding for unease.
uneasily is at index 12515
Saved the embedding for uneasily.
uneasiness is at index 125

In [109]:
################################################
# This cell will write out output embeddings   #
# for all the words in my vocabulary, using    #
# RoBERTa fine-tuned on wiki-103 training text.#
################################################

# THESE EMBEDDINGS GIVE A SCORE OF 1.0 FOR ALL WORD PAIRS ON THE
# SYNONYMY SCORING TASK: DO NOT USE!!!

# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_wiki-103/')

model = RobertaForMaskedLM.from_pretrained('./output_wiki-103/', config=config)

config = RobertaConfig.from_pretrained('./output_wiki-103/')
config.output_hidden_states = True
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_output_wiki-103.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
    v_embed = model.roberta.embeddings(v_tensor)
#     print(v_embed)
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0][0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1933,  0.2200,  0.0831,  ..., -0.0753, -0.0857, -0.1703],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for aback.
abashed is at index 4091
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.1475,  0.1399,  0.4928,  ...,  0.0418, -0.2483,  0.0081],
         [-0.1704,  0.3353,  0.0577,  ..., -0.3183, -0.2601,  0.1997],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for abashed.
abhor is at index 35350
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.5060,  0.0363, -0.0173,  ...,  0.1436,  0.1771,  0.1344],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding f

Saved the embedding for afflicted.
affronted is at index 11129
tensor([[[ 1.7678e-01, -2.3006e-02, -1.1993e-02,  ..., -1.1778e-01,
           8.3837e-02,  2.0007e-02],
         [ 9.8776e-02,  6.2653e-02,  4.3880e-01,  ..., -2.1984e-02,
           6.5535e-02, -5.6613e-01],
         [ 3.7706e-01, -2.8436e-01, -5.8371e-02,  ..., -3.1412e-01,
          -9.0653e-01,  6.5078e-02],
         [-7.6293e-02,  8.2298e-02,  3.7151e-01,  ...,  3.9176e-01,
          -5.2446e-01,  1.9090e-01],
         [ 2.3272e-01, -1.9380e-01, -1.1761e-01,  ...,  4.0675e-01,
          -6.5191e-04,  2.3365e-01]]], grad_fn=<NativeLayerNormBackward>)
Saved the embedding for affronted.
aflutter is at index 10
tensor([[[ 1.7678e-01, -2.3006e-02, -1.1993e-02,  ..., -1.1778e-01,
           8.3837e-02,  2.0007e-02],
         [-3.4255e-01, -1.8128e-01,  1.8741e-01,  ...,  2.3911e-01,
          -1.4931e-01,  4.2797e-01],
         [ 1.1907e-01,  4.8629e-01,  3.6980e-01,  ..., -2.7187e-01,
          -5.6087e-01,  4.5725e-01],
 

Saved the embedding for ambivalence.
ambivalent is at index 13569
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.0928, -0.1535,  0.0844,  ..., -0.2661, -0.4991,  0.3654],
         [ 0.0343,  1.1645, -0.0423,  ..., -0.1530, -0.1426, -0.3754],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for ambivalent.
amenable is at index 524
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1394,  0.0866,  0.1162,  ...,  0.1771,  0.0375,  0.1449],
         [ 0.1898,  0.2692,  0.0034,  ...,  0.0098, -0.2619,  0.1852],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for amenable.
amiable is at index 524
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1394,  0.0866,  0.1162,  ...,  0.1771,  0.0375,  0.1449],
         [ 0.0837

Saved the embedding for appallingly.
appeased is at index 44151
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1087, -0.5317, -0.2561,  ..., -0.1869,  0.2688, -0.0927],
         [-0.3413,  0.2097, -0.0392,  ..., -0.4061,  0.2151, -0.1586],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for appeased.
appeasing is at index 44151
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1087, -0.5317, -0.2561,  ..., -0.1869,  0.2688, -0.0927],
         [-0.2930,  0.3248, -0.1258,  ..., -0.4215,  0.5778, -0.0395],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for appeasing.
appreciative is at index 14137
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.4088, -0.3738,  0.3361,  ...,  0.4298,  0.5139,  0.0515],
         [

Saved the embedding for awakened.
aware is at index 2542
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.3167, -0.0695, -0.2455,  ..., -0.2955, -0.0768,  0.2597],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for aware.
awareness is at index 4199
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1428, -0.3666,  0.0266,  ..., -0.1353,  0.5158,  0.0881],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for awareness.
awe is at index 21531
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.2529, -0.3167,  0.2648,  ...,  0.1580,  0.1287, -0.2844],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for awe.
awed is at index 19267
tenso

...

Saved the embedding for unemotional.
unenthusiastic is at index 542
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1661, -0.4835,  0.2549,  ..., -0.1693, -0.2685, -0.4708],
         [ 0.0903, -0.2613,  0.2799,  ...,  0.0009,  0.2593,  0.6205],
         ...,
         [ 0.8461, -0.1502, -0.0947,  ..., -0.1917, -0.1337,  0.3315],
         [-0.0695,  0.3583, -0.0867,  ..., -0.2211, -0.0361,  0.1512],
         [ 0.2451, -0.1658, -0.0346,  ...,  0.3914,  0.1459,  0.1849]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for unenthusiastic.
unexcited is at index 39432
tensor([[[ 1.7678e-01, -2.3006e-02, -1.1993e-02,  ..., -1.1778e-01,
           8.3837e-02,  2.0007e-02],
         [-1.8515e-02,  3.0806e-01,  2.8911e-01,  ..., -1.3964e-01,
           3.6044e-02,  3.2512e-01],
         [ 2.5519e-01,  3.7928e-01, -1.6959e-01,  ...,  6.7259e-02,
          -1.1665e-01,  2.4678e-01],
         [ 5.0011e-01,  7.6923e-01,  4.9628e-02,  ...,  1.1828e-0

Saved the embedding for unsociable.
unspeaking is at index 542
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1661, -0.4835,  0.2549,  ..., -0.1693, -0.2685, -0.4708],
         [-0.3411, -0.4986, -0.4475,  ..., -0.4399, -0.4055, -0.4739],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for unspeaking.
unspoken is at index 542
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1661, -0.4835,  0.2549,  ..., -0.1693, -0.2685, -0.4708],
         [-0.5485,  0.1086, -0.3569,  ..., -0.5408, -0.0394, -0.3170],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for unspoken.
unstrung is at index 542
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1661, -0.4835,  0.2549,  ..., -0.1693, -0.2685, -0.4708],
         [ 0.0232, 

Saved the embedding for vexed.
vicious is at index 16339
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.2917,  0.5700, -0.0838,  ..., -0.0020, -0.1390,  0.3906],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for vicious.
victorious is at index 22518
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.0755, -0.5815,  0.3429,  ...,  0.0563,  0.0648,  0.1860],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for victorious.
vigilant is at index 17258
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.2403, -0.1699,  0.1110,  ..., -0.0956, -0.2690,  0.1613],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for vigilant.
vile is at in

tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.1258,  0.3327,  0.0427,  ...,  0.2645,  0.1700,  0.0413],
         [ 0.1057,  1.0936, -0.1933,  ..., -0.2768,  0.5266,  0.2954],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for whimpering.
whimsical is at index 29363
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.3293, -0.0334,  0.1540,  ...,  0.2871,  0.3565,  0.2077],
         [-0.3909,  0.5831,  0.0951,  ...,  0.2938,  0.2165, -0.0992],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for whimsical.
whisper is at index 37539
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.1171,  0.0063, -0.3555,  ..., -0.0726, -0.4253, -0.2073],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
    

Saved the embedding for wrathfully.
wrecked is at index 30090
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [ 0.1961,  0.1931,  0.2085,  ...,  0.2727, -0.1159, -0.1608],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for wrecked.
wretched is at index 42824
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.0876,  0.0728, -0.0089,  ...,  0.5445,  0.2370,  0.1607],
         [ 0.0924, -0.0295, -0.0129,  ...,  0.3057, -0.1149,  0.1891]]],
       grad_fn=<NativeLayerNormBackward>)
Saved the embedding for wretched.
wronged is at index 1593
tensor([[[ 0.1768, -0.0230, -0.0120,  ..., -0.1178,  0.0838,  0.0200],
         [-0.0088,  0.1192,  0.6745,  ..., -0.0280, -0.8351, -0.4304],
         [-0.1786,  0.2013,  0.4345,  ...,  0.3394, -0.5501,  0.1632],
         [ 0.1991, -0.1564, -0.0799,  ...,  0.3577, -0.0868,  0.2159]]],
       grad_fn=<Nat

In [10]:
################################################
# This cell will write out output embeddings   #
# at the word level for all the words in my    #
# vocabulary, using RoBERTa fine-tuned on      #
# wiki-103 training text.                      #
################################################

# THESE EMBEDDINGS GIVE A SCORE OF 1.0 FOR ALL WORD PAIRS ON THE
# SYNONYMY SCORING TASK: DO NOT USE!!!

# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_wiki-103/')

model = RobertaForMaskedLM.from_pretrained('./output_wiki-103/', config=config)

config = RobertaConfig.from_pretrained('./output_wiki-103/')
config.output_hidden_states = True
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_output-word_wiki-103.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
    v_embed = model.roberta.embeddings.word_embeddings(v_tensor)
#     print(v_embed)
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0][0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

NameError: name 'vocab' is not defined

In [16]:
################################################
# This cell will write out input embeddings    #
# for all the words in my                      #
# vocabulary, using RoBERTa fine-tuned once on #
# Common Crawl training text.                  #
################################################
# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_CC-aa/')

model = RobertaForMaskedLM.from_pretrained('./output_CC-aa/', config=config)

config = RobertaConfig.from_pretrained('./output_CC-aa/')
config.output_hidden_states = True
input_embeddings = model.get_input_embeddings()
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_input_CC_aa.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
#     print(input_embeddings_test(torch.LongTensor([v_tensor[0][1].item()])))
    v_embed = input_embeddings(torch.LongTensor([v_tensor[0][1].item()]))
#     for n in range(v_embed.size()[1])
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
Saved the embedding for aback.
abashed is at index 4091
Saved the embedding for abashed.
abhor is at index 35350
Saved the embedding for abhor.
abhorred is at index 35350
Saved the embedding for abhorred.
abhorrence is at index 35350
Saved the embedding for abhorrence.
abhorrent is at index 35350
Saved the embedding for abhorrent.
abominable is at index 4091
Saved the embedding for abominable.
abound is at index 32937
Saved the embedding for abound.
absent is at index 11640
Saved the embedding for absent.
absorbed is at index 22416
Saved the embedding for absorbed.
acceptance is at index 10502
Saved the embedding for acceptance.
accepted is at index 3903
Saved the embedding for accepted.
accepting is at index 8394
Saved the embedding for accepting.
accommodating is at index 33681
Saved the embedding for accommodating.
accomplished is at index 9370
Saved the embedding for accomplished.
accordant is at index 10170
Saved the embedding for accordant.
accursed is at 

Saved the embedding for awed.
awestruck is at index 19267
Saved the embedding for awestruck.
awful is at index 11522
Saved the embedding for awful.
awkward is at index 11789
Saved the embedding for awkward.
awkwardness is at index 11789
Saved the embedding for awkwardness.
axed is at index 18884
Saved the embedding for axed.
backhanded is at index 124
Saved the embedding for backhanded.
badly is at index 7340
Saved the embedding for badly.
baffle is at index 33139
Saved the embedding for baffle.
baffled is at index 33396
Saved the embedding for baffled.
baffling is at index 33139
Saved the embedding for baffling.
baked is at index 17241
Saved the embedding for baked.
banal is at index 2020
Saved the embedding for banal.
barking is at index 35828
Saved the embedding for barking.
bashful is at index 12882
Saved the embedding for bashful.
beaming is at index 28
Saved the embedding for beaming.
bearish is at index 4649
Saved the embedding for bearish.
beat is at index 1451
Saved the embedd

Saved the embedding for cheeky.
cheered is at index 18643
Saved the embedding for cheered.
cheerful is at index 33928
Saved the embedding for cheerful.
cheering is at index 16765
Saved the embedding for cheering.
cheerless is at index 9450
Saved the embedding for cheerless.
cheery is at index 5851
Saved the embedding for cheery.
cheesy is at index 36331
Saved the embedding for cheesy.
chesty is at index 7050
Saved the embedding for chesty.
chide is at index 1855
Saved the embedding for chide.
chiding is at index 1855
Saved the embedding for chiding.
childish is at index 40531
Saved the embedding for childish.
childishly is at index 920
Saved the embedding for childishly.
childlike is at index 920
Saved the embedding for childlike.
chill is at index 13146
Saved the embedding for chill.
chilled is at index 32338
Saved the embedding for chilled.
chilling is at index 22577
Saved the embedding for chilling.
chipper is at index 1855
Saved the embedding for chipper.
chirpy is at index 1855
Sa

Saved the embedding for cross.
crotchety is at index 11398
Saved the embedding for crotchety.
crude is at index 2976
Saved the embedding for crude.
cruel is at index 15939
Saved the embedding for cruel.
crushed is at index 14045
Saved the embedding for crushed.
cry is at index 8930
Saved the embedding for cry.
crying is at index 9701
Saved the embedding for crying.
cryptic is at index 35916
Saved the embedding for cryptic.
culpable is at index 29410
Saved the embedding for culpable.
cunning is at index 41526
Saved the embedding for cunning.
curios is at index 5350
Saved the embedding for curios.
curiosity is at index 20610
Saved the embedding for curiosity.
curious is at index 10691
Saved the embedding for curious.
cutting is at index 3931
Saved the embedding for cutting.
cynic is at index 40240
Saved the embedding for cynic.
cynical is at index 27566
Saved the embedding for cynical.
cynicism is at index 39245
Saved the embedding for cynicism.
dalliance is at index 385
Saved the embedd

Saved the embedding for disgruntlement.
disgust is at index 30883
Saved the embedding for disgust.
disgusted is at index 32759
Saved the embedding for disgusted.
disgustedly is at index 32759
Saved the embedding for disgustedly.
disgusting is at index 21096
Saved the embedding for disgusting.
disheartened is at index 2982
Saved the embedding for disheartened.
dishonest is at index 27820
Saved the embedding for dishonest.
disillusioned is at index 33447
Saved the embedding for disillusioned.
disinclined is at index 2982
Saved the embedding for disinclined.
disingenuous is at index 39622
Saved the embedding for disingenuous.
disinterest is at index 2982
Saved the embedding for disinterest.
disinterested is at index 2982
Saved the embedding for disinterested.
disjointed is at index 2982
Saved the embedding for disjointed.
dislike is at index 28101
Saved the embedding for dislike.
disliked is at index 40891
Saved the embedding for disliked.
disliking is at index 19131
Saved the embedding f

...

Saved the embedding for relief.
relieved is at index 15126
Saved the embedding for relieved.
relived is at index 6258
Saved the embedding for relived.
reluctant is at index 11923
Saved the embedding for reluctant.
reluctantly is at index 33146
Saved the embedding for reluctantly.
remorse is at index 23312
Saved the embedding for remorse.
remorseful is at index 23312
Saved the embedding for remorseful.
repelled is at index 25633
Saved the embedding for repelled.
repressed is at index 2851
Saved the embedding for repressed.
reproach is at index 2851
Saved the embedding for reproach.
reproachful is at index 2851
Saved the embedding for reproachful.
repugnance is at index 2851
Saved the embedding for repugnance.
repugnant is at index 2851
Saved the embedding for repugnant.
repulsed is at index 2851
Saved the embedding for repulsed.
repulsion is at index 2851
Saved the embedding for repulsion.
resent is at index 31379
Saved the embedding for resent.
resentful is at index 31379
Saved the emb

Saved the embedding for sigh.
silenced is at index 30125
Saved the embedding for silenced.
silent is at index 8454
Saved the embedding for silent.
silliness is at index 38052
Saved the embedding for silliness.
silly is at index 15470
Saved the embedding for silly.
simmering is at index 25726
Saved the embedding for simmering.
simper is at index 16207
Saved the embedding for simper.
simpering is at index 16207
Saved the embedding for simpering.
simple is at index 2007
Saved the embedding for simple.
simplicity is at index 25342
Saved the embedding for simplicity.
sincere is at index 19255
Saved the embedding for sincere.
sinful is at index 44364
Saved the embedding for sinful.
singing is at index 6970
Saved the embedding for singing.
sinister is at index 27570
Saved the embedding for sinister.
sinisterly is at index 27570
Saved the embedding for sinisterly.
sizing is at index 39328
Saved the embedding for sizing.
skeptic is at index 42386
Saved the embedding for skeptic.
skeptical is at

Saved the embedding for submissive.
suffering is at index 3606
Saved the embedding for suffering.
suggestive is at index 38907
Saved the embedding for suggestive.
sulking is at index 26648
Saved the embedding for sulking.
sulky is at index 26648
Saved the embedding for sulky.
sullen is at index 2628
Saved the embedding for sullen.
sullenness is at index 2628
Saved the embedding for sullenness.
sunny is at index 5419
Saved the embedding for sunny.
superior is at index 10295
Saved the embedding for superior.
superiority is at index 32951
Saved the embedding for superiority.
suppressed is at index 31683
Saved the embedding for suppressed.
suppressing is at index 38919
Saved the embedding for suppressing.
suppression is at index 25276
Saved the embedding for suppression.
sure is at index 686
Saved the embedding for sure.
surly is at index 8113
Saved the embedding for surly.
surprise is at index 2755
Saved the embedding for surprise.
surprised is at index 3911
Saved the embedding for surpri

Saved the embedding for undecided.
underhanded is at index 223
Saved the embedding for underhanded.
understanding is at index 2969
Saved the embedding for understanding.
undesirable is at index 39028
Saved the embedding for undesirable.
unease is at index 12515
Saved the embedding for unease.
uneasily is at index 12515
Saved the embedding for uneasily.
uneasiness is at index 12515
Saved the embedding for uneasiness.
uneasy is at index 29569
Saved the embedding for uneasy.
unemotional is at index 542
Saved the embedding for unemotional.
unenthusiastic is at index 542
Saved the embedding for unenthusiastic.
unexcited is at index 39432
Saved the embedding for unexcited.
unexpected is at index 7152
Saved the embedding for unexpected.
unfamiliar is at index 21942
Saved the embedding for unfamiliar.
unfathomable is at index 9515
Saved the embedding for unfathomable.
unfazed is at index 9515
Saved the embedding for unfazed.
unfeeling is at index 9515
Saved the embedding for unfeeling.
unfocus

wry is at index 885
Saved the embedding for wry.
yawn is at index 39654
Saved the embedding for yawn.
yawning is at index 39654
Saved the embedding for yawning.
yearning is at index 76
Saved the embedding for yearning.
yell is at index 28930
Saved the embedding for yell.
yelling is at index 16600
Saved the embedding for yelling.
yielding is at index 25438
Saved the embedding for yielding.
yuck is at index 1423
Saved the embedding for yuck.
zany is at index 992
Saved the embedding for zany.
zealous is at index 992
Saved the embedding for zealous.
zen is at index 992
Saved the embedding for zen.
zoned is at index 992
Saved the embedding for zoned.


In [17]:
####################################################
# This cell will write out output embeddings       #
# for all the words in my vocabulary, using RoBERTa#
# fine-tuned once on Common Crawl training text.   #
####################################################

# THESE EMBEDDINGS GIVE A SCORE OF 1.0 FOR ALL WORD PAIRS ON THE
# SYNONYMY SCORING TASK: DO NOT USE!!!

# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_CC-aa/')

model = RobertaForMaskedLM.from_pretrained('./output_CC-aa/', config=config)

config = RobertaConfig.from_pretrained('./output_CC-aa/')
config.output_hidden_states = True
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_output_CC_aa.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
    v_embed = model.roberta.embeddings(v_tensor)
#     print(v_embed)
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0][0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
Saved the embedding for aback.
abashed is at index 4091
Saved the embedding for abashed.
abhor is at index 35350
Saved the embedding for abhor.
abhorred is at index 35350
Saved the embedding for abhorred.
abhorrence is at index 35350
Saved the embedding for abhorrence.
abhorrent is at index 35350
Saved the embedding for abhorrent.
abominable is at index 4091
Saved the embedding for abominable.
abound is at index 32937
Saved the embedding for abound.
absent is at index 11640
Saved the embedding for absent.
absorbed is at index 22416
Saved the embedding for absorbed.
acceptance is at index 10502
Saved the embedding for acceptance.
accepted is at index 3903
Saved the embedding for accepted.
accepting is at index 8394
Saved the embedding for accepting.
accommodating is at index 33681
Saved the embedding for accommodating.
accomplished is at index 9370
Saved the embedding for accomplished.
accordant is at index 10170
Saved the embedding for accordant.
accursed is at 

Saved the embedding for benevolence.
benevolent is at index 43186
Saved the embedding for benevolent.
benumbed is at index 21576
Saved the embedding for benumbed.
berate is at index 14719
Saved the embedding for berate.
berating is at index 14719
Saved the embedding for berating.
bereaved is at index 17738
Saved the embedding for bereaved.
bereft is at index 17738
Saved the embedding for bereft.
beseeching is at index 9988
Saved the embedding for beseeching.
bested is at index 275
Saved the embedding for bested.
betrayal is at index 26760
Saved the embedding for betrayal.
betrayed is at index 26913
Saved the embedding for betrayed.
bewildered is at index 33304
Saved the embedding for bewildered.
bewilderment is at index 33304
Saved the embedding for bewilderment.
bi is at index 4003
Saved the embedding for bi.
bilious is at index 31617
Saved the embedding for bilious.
bit is at index 828
Saved the embedding for bit.
biting is at index 25609
Saved the embedding for biting.
bitter is at 

Saved the embedding for compulsive.
concealed is at index 17180
Saved the embedding for concealed.
conceding is at index 24647
Saved the embedding for conceding.
conceited is at index 21177
Saved the embedding for conceited.
concentrated is at index 15450
Saved the embedding for concentrated.
concentrating is at index 28619
Saved the embedding for concentrating.
concentration is at index 11772
Saved the embedding for concentration.
concern is at index 2212
Saved the embedding for concern.
concerned is at index 2273
Saved the embedding for concerned.
conciliatory is at index 10146
Saved the embedding for conciliatory.
conclusive is at index 37847
Saved the embedding for conclusive.
condemning is at index 21856
Saved the embedding for condemning.
condescending is at index 40742
Saved the embedding for condescending.
condoling is at index 35279
Saved the embedding for condoling.
confidence is at index 2123
Saved the embedding for confidence.
confident is at index 3230
Saved the embedding 

demoralized is at index 36810
Saved the embedding for demoralized.
demure is at index 4410
Saved the embedding for demure.
denied is at index 2296
Saved the embedding for denied.
denouncing is at index 32439
Saved the embedding for denouncing.
depleted is at index 26391
Saved the embedding for depleted.
deplorable is at index 28156
Saved the embedding for deplorable.
deprecating is at index 8273
Saved the embedding for deprecating.
depressed is at index 16658
Saved the embedding for depressed.
depression is at index 6943
Saved the embedding for depression.
deprived is at index 22632
Saved the embedding for deprived.
deranged is at index 1935
Saved the embedding for deranged.
derision is at index 1935
Saved the embedding for derision.
derisive is at index 1935
Saved the embedding for derisive.
derogatory is at index 30971
Saved the embedding for derogatory.
desire is at index 4724
Saved the embedding for desire.
desiring is at index 2694
Saved the embedding for desiring.
desirous is at 

Saved the embedding for diverted.
dodgy is at index 25744
Saved the embedding for dodgy.
doleful is at index 109
Saved the embedding for doleful.
doltish is at index 385
Saved the embedding for doltish.
dominant is at index 7353
Saved the embedding for dominant.
dominating is at index 17349
Saved the embedding for dominating.
domineering is at index 13567
Saved the embedding for domineering.
done is at index 626
Saved the embedding for done.
doomed is at index 23326
Saved the embedding for doomed.
dopey is at index 32331
Saved the embedding for dopey.
doting is at index 385
Saved the embedding for doting.
doubt is at index 2980
Saved the embedding for doubt.
doubter is at index 26463
Saved the embedding for doubter.
doubtful is at index 26645
Saved the embedding for doubtful.
doubtfully is at index 2980
Saved the embedding for doubtfully.
doubtfulness is at index 2980
Saved the embedding for doubtfulness.
doubting is at index 26463
Saved the embedding for doubting.
dour is at index 385

...

questionable is at index 12474
Saved the embedding for questionable.
questioning is at index 8026
Saved the embedding for questioning.
questioningly is at index 864
Saved the embedding for questioningly.
quiet is at index 5128
Saved the embedding for quiet.
quietness is at index 5128
Saved the embedding for quietness.
quilt is at index 2677
Saved the embedding for quilt.
quirky is at index 22364
Saved the embedding for quirky.
quizzical is at index 29316
Saved the embedding for quizzical.
rabid is at index 39660
Saved the embedding for rabid.
racked is at index 20208
Saved the embedding for racked.
radiant is at index 35787
Saved the embedding for radiant.
rage is at index 14706
Saved the embedding for rage.
raged is at index 31927
Saved the embedding for raged.
ragged is at index 910
Saved the embedding for ragged.
raging is at index 23333
Saved the embedding for raging.
rancorous is at index 21560
Saved the embedding for rancorous.
randy is at index 910
Saved the embedding for randy.

shocked is at index 6649
Saved the embedding for shocked.
shocking is at index 8777
Saved the embedding for shocking.
shockingly is at index 36804
Saved the embedding for shockingly.
shook is at index 14774
Saved the embedding for shook.
shout is at index 18066
Saved the embedding for shout.
shouting is at index 14487
Saved the embedding for shouting.
shrewd is at index 36943
Saved the embedding for shrewd.
shy is at index 9152
Saved the embedding for shy.
shyness is at index 9152
Saved the embedding for shyness.
sick is at index 4736
Saved the embedding for sick.
sicken is at index 579
Saved the embedding for sicken.
sickened is at index 4736
Saved the embedding for sickened.
sigh is at index 27305
Saved the embedding for sigh.
silenced is at index 30125
Saved the embedding for silenced.
silent is at index 8454
Saved the embedding for silent.
silliness is at index 38052
Saved the embedding for silliness.
silly is at index 15470
Saved the embedding for silly.
simmering is at index 2572

Saved the embedding for stressed.
stricken is at index 35876
Saved the embedding for stricken.
strict is at index 8414
Saved the embedding for strict.
strong is at index 670
Saved the embedding for strong.
struck is at index 2322
Saved the embedding for struck.
stubborn is at index 20476
Saved the embedding for stubborn.
stubbornness is at index 20476
Saved the embedding for stubbornness.
studious is at index 15863
Saved the embedding for studious.
studying is at index 7739
Saved the embedding for studying.
stumped is at index 1690
Saved the embedding for stumped.
stung is at index 1690
Saved the embedding for stung.
stunned is at index 12144
Saved the embedding for stunned.
stupefaction is at index 1690
Saved the embedding for stupefaction.
stupefied is at index 1690
Saved the embedding for stupefied.
stupefy is at index 1690
Saved the embedding for stupefy.
stupid is at index 12103
Saved the embedding for stupid.
stuporous is at index 1690
Saved the embedding for stuporous.
suave is 

unamused is at index 542
Saved the embedding for unamused.
unappreciative is at index 542
Saved the embedding for unappreciative.
unapproachable is at index 542
Saved the embedding for unapproachable.
unassertive is at index 542
Saved the embedding for unassertive.
unassuming is at index 542
Saved the embedding for unassuming.
unaware is at index 14021
Saved the embedding for unaware.
unbelief is at index 46646
Saved the embedding for unbelief.
unbelievable is at index 14011
Saved the embedding for unbelievable.
unbelieving is at index 46646
Saved the embedding for unbelieving.
unbothered is at index 542
Saved the embedding for unbothered.
uncaring is at index 16511
Saved the embedding for uncaring.
uncertain is at index 9684
Saved the embedding for uncertain.
uncertainly is at index 9684
Saved the embedding for uncertainly.
uncertainty is at index 4983
Saved the embedding for uncertainty.
uncivil is at index 16511
Saved the embedding for uncivil.
uncomfortable is at index 9800
Saved t

Saved the embedding for weird.
welcome is at index 2814
Saved the embedding for welcome.
welcoming is at index 10423
Saved the embedding for welcoming.
whatever is at index 3046
Saved the embedding for whatever.
whimpering is at index 31754
Saved the embedding for whimpering.
whimsical is at index 29363
Saved the embedding for whimsical.
whisper is at index 37539
Saved the embedding for whisper.
whistle is at index 16867
Saved the embedding for whistle.
white is at index 1104
Saved the embedding for white.
wicked is at index 28418
Saved the embedding for wicked.
wild is at index 3418
Saved the embedding for wild.
willful is at index 40960
Saved the embedding for willful.
willing is at index 2882
Saved the embedding for willing.
wily is at index 885
Saved the embedding for wily.
wink is at index 39422
Saved the embedding for wink.
wired is at index 26977
Saved the embedding for wired.
wishful is at index 2813
Saved the embedding for wishful.
wistful is at index 885
Saved the embedding f

In [18]:
################################################
# This cell will write out input embeddings    #
# for all the words in my                      #
# vocabulary, using RoBERTa fine-tuned twice on#
# Common Crawl training text.                  #
# !!!USE INPUT EMBEDDINGS!!!!!                 #
################################################
# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_CC-ab/')

model = RobertaForMaskedLM.from_pretrained('./output_CC-ab/', config=config)

config = RobertaConfig.from_pretrained('./output_CC-ab/')
config.output_hidden_states = True
input_embeddings = model.get_input_embeddings()
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_input_CC_ab.txt'
for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
#     print(input_embeddings_test(torch.LongTensor([v_tensor[0][1].item()])))
    v_embed = input_embeddings(torch.LongTensor([v_tensor[0][1].item()]))
#     for n in range(v_embed.size()[1])
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
Saved the embedding for aback.
abashed is at index 4091
Saved the embedding for abashed.
abhor is at index 35350
Saved the embedding for abhor.
abhorred is at index 35350
Saved the embedding for abhorred.
abhorrence is at index 35350
Saved the embedding for abhorrence.
abhorrent is at index 35350
Saved the embedding for abhorrent.
abominable is at index 4091
Saved the embedding for abominable.
abound is at index 32937
Saved the embedding for abound.
absent is at index 11640
Saved the embedding for absent.
absorbed is at index 22416
Saved the embedding for absorbed.
acceptance is at index 10502
Saved the embedding for acceptance.
accepted is at index 3903
Saved the embedding for accepted.
accepting is at index 8394
Saved the embedding for accepting.
accommodating is at index 33681
Saved the embedding for accommodating.
accomplished is at index 9370
Saved the embedding for accomplished.
accordant is at index 10170
Saved the embedding for accordant.
accursed is at 

Saved the embedding for awkward.
awkwardness is at index 11789
Saved the embedding for awkwardness.
axed is at index 18884
Saved the embedding for axed.
backhanded is at index 124
Saved the embedding for backhanded.
badly is at index 7340
Saved the embedding for badly.
baffle is at index 33139
Saved the embedding for baffle.
baffled is at index 33396
Saved the embedding for baffled.
baffling is at index 33139
Saved the embedding for baffling.
baked is at index 17241
Saved the embedding for baked.
banal is at index 2020
Saved the embedding for banal.
barking is at index 35828
Saved the embedding for barking.
bashful is at index 12882
Saved the embedding for bashful.
beaming is at index 28
Saved the embedding for beaming.
bearish is at index 4649
Saved the embedding for bearish.
beat is at index 1451
Saved the embedding for beat.
beaten is at index 6432
Saved the embedding for beaten.
bedeviled is at index 3267
Saved the embedding for bedeviled.
befuddled is at index 28
Saved the embeddi

Saved the embedding for chiding.
childish is at index 40531
Saved the embedding for childish.
childishly is at index 920
Saved the embedding for childishly.
childlike is at index 920
Saved the embedding for childlike.
chill is at index 13146
Saved the embedding for chill.
chilled is at index 32338
Saved the embedding for chilled.
chilling is at index 22577
Saved the embedding for chilling.
chipper is at index 1855
Saved the embedding for chipper.
chirpy is at index 1855
Saved the embedding for chirpy.
choleric is at index 1855
Saved the embedding for choleric.
chortling is at index 1855
Saved the embedding for chortling.
chuckle is at index 37496
Saved the embedding for chuckle.
chuckling is at index 34600
Saved the embedding for chuckling.
churlish is at index 1855
Saved the embedding for churlish.
circumspect is at index 38529
Saved the embedding for circumspect.
clamorous is at index 24045
Saved the embedding for clamorous.
clash is at index 6064
Saved the embedding for clash.
clear

Saved the embedding for cruel.
crushed is at index 14045
Saved the embedding for crushed.
cry is at index 8930
Saved the embedding for cry.
crying is at index 9701
Saved the embedding for crying.
cryptic is at index 35916
Saved the embedding for cryptic.
culpable is at index 29410
Saved the embedding for culpable.
cunning is at index 41526
Saved the embedding for cunning.
curios is at index 5350
Saved the embedding for curios.
curiosity is at index 20610
Saved the embedding for curiosity.
curious is at index 10691
Saved the embedding for curious.
cutting is at index 3931
Saved the embedding for cutting.
cynic is at index 40240
Saved the embedding for cynic.
cynical is at index 27566
Saved the embedding for cynical.
cynicism is at index 39245
Saved the embedding for cynicism.
dalliance is at index 385
Saved the embedding for dalliance.
dandy is at index 385
Saved the embedding for dandy.
dangerous is at index 2702
Saved the embedding for dangerous.
darkly is at index 2933
Saved the embe

disgusting is at index 21096
Saved the embedding for disgusting.
disheartened is at index 2982
Saved the embedding for disheartened.
dishonest is at index 27820
Saved the embedding for dishonest.
disillusioned is at index 33447
Saved the embedding for disillusioned.
disinclined is at index 2982
Saved the embedding for disinclined.
disingenuous is at index 39622
Saved the embedding for disingenuous.
disinterest is at index 2982
Saved the embedding for disinterest.
disinterested is at index 2982
Saved the embedding for disinterested.
disjointed is at index 2982
Saved the embedding for disjointed.
dislike is at index 28101
Saved the embedding for dislike.
disliked is at index 40891
Saved the embedding for disliked.
disliking is at index 19131
Saved the embedding for disliking.
dismal is at index 23446
Saved the embedding for dismal.
disman is at index 2982
Saved the embedding for disman.
dismay is at index 22135
Saved the embedding for dismay.
dismayed is at index 22135
Saved the embeddin

...

Saved the embedding for pissed.
pitiable is at index 8516
Saved the embedding for pitiable.
pitiful is at index 8516
Saved the embedding for pitiful.
pity is at index 31373
Saved the embedding for pity.
pitying is at index 31373
Saved the embedding for pitying.
placated is at index 15155
Saved the embedding for placated.
placation is at index 15155
Saved the embedding for placation.
placid is at index 15155
Saved the embedding for placid.
plain is at index 10798
Saved the embedding for plain.
plaintive is at index 46560
Saved the embedding for plaintive.
planning is at index 1884
Saved the embedding for planning.
playful is at index 23317
Saved the embedding for playful.
playfully is at index 310
Saved the embedding for playfully.
pleading is at index 17532
Saved the embedding for pleading.
pleasant is at index 16219
Saved the embedding for pleasant.
pleased is at index 4343
Saved the embedding for pleased.
pleasing is at index 25234
Saved the embedding for pleasing.
pleasurable is at 

Saved the embedding for resistant.
resistent is at index 11942
Saved the embedding for resistent.
resisting is at index 18907
Saved the embedding for resisting.
resolute is at index 5032
Saved the embedding for resolute.
resolved is at index 8179
Saved the embedding for resolved.
responsive is at index 20666
Saved the embedding for responsive.
restful is at index 1079
Saved the embedding for restful.
resting is at index 18403
Saved the embedding for resting.
restless is at index 36844
Saved the embedding for restless.
restlessness is at index 1079
Saved the embedding for restlessness.
restrained is at index 25063
Saved the embedding for restrained.
restraint is at index 20219
Saved the embedding for restraint.
retaliating is at index 18570
Saved the embedding for retaliating.
retaliatory is at index 18570
Saved the embedding for retaliatory.
rethinking is at index 769
Saved the embedding for rethinking.
reticence is at index 5494
Saved the embedding for reticence.
reticent is at index 

Saved the embedding for slothful.
slow is at index 2635
Saved the embedding for slow.
sluggish is at index 16642
Saved the embedding for sluggish.
sly is at index 40568
Saved the embedding for sly.
smarmy is at index 5278
Saved the embedding for smarmy.
smart is at index 2793
Saved the embedding for smart.
smashed is at index 13263
Saved the embedding for smashed.
smile is at index 6675
Saved the embedding for smile.
smiley is at index 6675
Saved the embedding for smiley.
smiling is at index 12382
Saved the embedding for smiling.
smirk is at index 5278
Saved the embedding for smirk.
smirking is at index 44414
Saved the embedding for smirking.
smoldering is at index 5278
Saved the embedding for smoldering.
smooching is at index 5278
Saved the embedding for smooching.
smooth is at index 6921
Saved the embedding for smooth.
smug is at index 41283
Saved the embedding for smug.
smugness is at index 41283
Saved the embedding for smugness.
snake is at index 16173
Saved the embedding for snake

Saved the embedding for suspecting.
suspense is at index 31803
Saved the embedding for suspense.
suspicion is at index 8551
Saved the embedding for suspicion.
suspicious is at index 7775
Saved the embedding for suspicious.
suspiciously is at index 7775
Saved the embedding for suspiciously.
suspiciousness is at index 7775
Saved the embedding for suspiciousness.
swaggering is at index 3514
Saved the embedding for swaggering.
swearing is at index 21854
Saved the embedding for swearing.
sympathetic is at index 22869
Saved the embedding for sympathetic.
sympathizing is at index 19023
Saved the embedding for sympathizing.
sympathy is at index 16554
Saved the embedding for sympathy.
taciturn is at index 36502
Saved the embedding for taciturn.
talkative is at index 1067
Saved the embedding for talkative.
talking is at index 1686
Saved the embedding for talking.
tantalized is at index 33496
Saved the embedding for tantalized.
tart is at index 27468
Saved the embedding for tart.
tasteful is at i

Saved the embedding for uninformed.
uninspired is at index 542
Saved the embedding for uninspired.
uninterested is at index 542
Saved the embedding for uninterested.
uninvolved is at index 542
Saved the embedding for uninvolved.
unique is at index 2216
Saved the embedding for unique.
unlikeable is at index 7328
Saved the embedding for unlikeable.
unmoved is at index 30780
Saved the embedding for unmoved.
unnerved is at index 31550
Saved the embedding for unnerved.
unpleasant is at index 26262
Saved the embedding for unpleasant.
unprepared is at index 35578
Saved the embedding for unprepared.
unquiet is at index 542
Saved the embedding for unquiet.
unreactive is at index 21153
Saved the embedding for unreactive.
unresolved is at index 29909
Saved the embedding for unresolved.
unrestrained is at index 12254
Saved the embedding for unrestrained.
unruffled is at index 542
Saved the embedding for unruffled.
unsatisfied is at index 36010
Saved the embedding for unsatisfied.
unsettled is at i

In [19]:
####################################################
# This cell will write out output embeddings       #
# for all the words in my vocabulary, using RoBERTa#
# fine-tuned twice on Common Crawl training text.  #
####################################################

# THESE EMBEDDINGS GIVE A SCORE OF 1.0 FOR ALL WORD PAIRS ON THE
# SYNONYMY SCORING TASK: DO NOT USE!!!

# Load pre-trained model tokenizer (vocabulary)
tokenizer = RobertaTokenizer.from_pretrained('./output_CC-ab/')
model = RobertaForMaskedLM.from_pretrained('./output_CC-ab/', config=config)
config = RobertaConfig.from_pretrained('./output_CC-ab/')
config.output_hidden_states = True
embeddings_file = '/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab/roberta_output_CC_ab.txt'

for v in vocab:
    v_tensor = torch.tensor([tokenizer.encode(v)])
    # Print the index of the test word.
    print(f'{v} is at index {v_tensor[0][1].item()}')
    v_embed = model.roberta.embeddings(v_tensor)
#     print(v_embed)
    try:
        with open(embeddings_file, 'a') as f:
            f.write(v)
            for value in v_embed[0][0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {v}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

aback is at index 36347
Saved the embedding for aback.
abashed is at index 4091
Saved the embedding for abashed.
abhor is at index 35350
Saved the embedding for abhor.
abhorred is at index 35350
Saved the embedding for abhorred.
abhorrence is at index 35350
Saved the embedding for abhorrence.
abhorrent is at index 35350
Saved the embedding for abhorrent.
abominable is at index 4091
Saved the embedding for abominable.
abound is at index 32937
Saved the embedding for abound.
absent is at index 11640
Saved the embedding for absent.
absorbed is at index 22416
Saved the embedding for absorbed.
acceptance is at index 10502
Saved the embedding for acceptance.
accepted is at index 3903
Saved the embedding for accepted.
accepting is at index 8394
Saved the embedding for accepting.
accommodating is at index 33681
Saved the embedding for accommodating.
accomplished is at index 9370
Saved the embedding for accomplished.
accordant is at index 10170
Saved the embedding for accordant.
accursed is at 

Saved the embedding for avoiding.
awaiting is at index 10254
Saved the embedding for awaiting.
awakened is at index 40593
Saved the embedding for awakened.
aware is at index 2542
Saved the embedding for aware.
awareness is at index 4199
Saved the embedding for awareness.
awe is at index 21531
Saved the embedding for awe.
awed is at index 19267
Saved the embedding for awed.
awestruck is at index 19267
Saved the embedding for awestruck.
awful is at index 11522
Saved the embedding for awful.
awkward is at index 11789
Saved the embedding for awkward.
awkwardness is at index 11789
Saved the embedding for awkwardness.
axed is at index 18884
Saved the embedding for axed.
backhanded is at index 124
Saved the embedding for backhanded.
badly is at index 7340
Saved the embedding for badly.
baffle is at index 33139
Saved the embedding for baffle.
baffled is at index 33396
Saved the embedding for baffled.
baffling is at index 33139
Saved the embedding for baffling.
baked is at index 17241
Saved the

Saved the embedding for chill.
chilled is at index 32338
Saved the embedding for chilled.
chilling is at index 22577
Saved the embedding for chilling.
chipper is at index 1855
Saved the embedding for chipper.
chirpy is at index 1855
Saved the embedding for chirpy.
choleric is at index 1855
Saved the embedding for choleric.
chortling is at index 1855
Saved the embedding for chortling.
chuckle is at index 37496
Saved the embedding for chuckle.
chuckling is at index 34600
Saved the embedding for chuckling.
churlish is at index 1855
Saved the embedding for churlish.
circumspect is at index 38529
Saved the embedding for circumspect.
clamorous is at index 24045
Saved the embedding for clamorous.
clash is at index 6064
Saved the embedding for clash.
clear is at index 699
Saved the embedding for clear.
clenched is at index 44646
Saved the embedding for clenched.
clever is at index 13074
Saved the embedding for clever.
close is at index 593
Saved the embedding for close.
closed is at index 1367

Saved the embedding for defenseless.
defensive is at index 2465
Saved the embedding for defensive.
defiance is at index 25442
Saved the embedding for defiance.
defiant is at index 23802
Saved the embedding for defiant.
deflated is at index 3816
Saved the embedding for deflated.
degage is at index 31295
Saved the embedding for degage.
degrading is at index 36892
Saved the embedding for degrading.
dejected is at index 263
Saved the embedding for dejected.
dejection is at index 263
Saved the embedding for dejection.
deliberate is at index 14775
Saved the embedding for deliberate.
deliberating is at index 21614
Saved the embedding for deliberating.
delight is at index 13213
Saved the embedding for delight.
delighted is at index 7808
Saved the embedding for delighted.
delightful is at index 24897
Saved the embedding for delightful.
delirious is at index 2424
Saved the embedding for delirious.
delirium is at index 2424
Saved the embedding for delirium.
delude is at index 2424
Saved the embed

Saved the embedding for doubt.
doubter is at index 26463
Saved the embedding for doubter.
doubtful is at index 26645
Saved the embedding for doubtful.
doubtfully is at index 2980
Saved the embedding for doubtfully.
doubtfulness is at index 2980
Saved the embedding for doubtfulness.
doubting is at index 26463
Saved the embedding for doubting.
dour is at index 385
Saved the embedding for dour.
down is at index 159
Saved the embedding for down.
downcast is at index 159
Saved the embedding for downcast.
downhearted is at index 159
Saved the embedding for downhearted.
downheartedness is at index 159
Saved the embedding for downheartedness.
downtrodden is at index 29407
Saved the embedding for downtrodden.
dozing is at index 109
Saved the embedding for dozing.
drained is at index 23544
Saved the embedding for drained.
dramatic is at index 5386
Saved the embedding for dramatic.
drawn is at index 4777
Saved the embedding for drawn.
dread is at index 24506
Saved the embedding for dread.
dreadfu

...

Saved the embedding for poised.
polite is at index 24908
Saved the embedding for polite.
pompous is at index 34415
Saved the embedding for pompous.
ponder is at index 31930
Saved the embedding for ponder.
pondering is at index 13362
Saved the embedding for pondering.
pooping is at index 4202
Saved the embedding for pooping.
pop is at index 3495
Saved the embedding for pop.
posing is at index 12681
Saved the embedding for posing.
positive is at index 1313
Saved the embedding for positive.
positivity is at index 8593
Saved the embedding for positivity.
possibly is at index 3544
Saved the embedding for possibly.
pout is at index 181
Saved the embedding for pout.
pouting is at index 181
Saved the embedding for pouting.
pouty is at index 181
Saved the embedding for pouty.
powerful is at index 2247
Saved the embedding for powerful.
powerless is at index 33128
Saved the embedding for powerless.
pranking is at index 3349
Saved the embedding for pranking.
precarious is at index 27180
Saved the 

Saved the embedding for sassy.
sated is at index 579
Saved the embedding for sated.
satiated is at index 4005
Saved the embedding for satiated.
satirical is at index 33937
Saved the embedding for satirical.
satisfaction is at index 11658
Saved the embedding for satisfaction.
satisfied is at index 10028
Saved the embedding for satisfied.
satisfy is at index 15332
Saved the embedding for satisfy.
saturnine is at index 4005
Saved the embedding for saturnine.
saucy is at index 2241
Saved the embedding for saucy.
savage is at index 32264
Saved the embedding for savage.
scandalized is at index 4220
Saved the embedding for scandalized.
scare is at index 13207
Saved the embedding for scare.
scared is at index 8265
Saved the embedding for scared.
scary is at index 10222
Saved the embedding for scary.
scattered is at index 12827
Saved the embedding for scattered.
schadenfreude is at index 8447
Saved the embedding for schadenfreude.
scheming is at index 30315
Saved the embedding for scheming.
sco

Saved the embedding for solitary.
solitude is at index 41813
Saved the embedding for solitude.
somber is at index 16487
Saved the embedding for somber.
somberly is at index 16487
Saved the embedding for somberly.
somnolent is at index 16487
Saved the embedding for somnolent.
soothed is at index 98
Saved the embedding for soothed.
sore is at index 12867
Saved the embedding for sore.
sorrow is at index 26130
Saved the embedding for sorrow.
sorrowful is at index 26130
Saved the embedding for sorrowful.
sorry is at index 6661
Saved the embedding for sorry.
sour is at index 16933
Saved the embedding for sour.
spaced is at index 42926
Saved the embedding for spaced.
spacing is at index 39152
Saved the embedding for spacing.
spastic is at index 2292
Saved the embedding for spastic.
speaking is at index 2686
Saved the embedding for speaking.
specious is at index 12002
Saved the embedding for specious.
speculative is at index 21779
Saved the embedding for speculative.
speechless is at index 190

Saved the embedding for tranquil.
tranquility is at index 36474
Saved the embedding for tranquility.
transfixed is at index 30387
Saved the embedding for transfixed.
traumatized is at index 25178
Saved the embedding for traumatized.
trembling is at index 44912
Saved the embedding for trembling.
trepid is at index 6110
Saved the embedding for trepid.
trepidation is at index 6110
Saved the embedding for trepidation.
trickster is at index 7610
Saved the embedding for trickster.
tricky is at index 12792
Saved the embedding for tricky.
triumphant is at index 32025
Saved the embedding for triumphant.
troubled is at index 9895
Saved the embedding for troubled.
troublesome is at index 34056
Saved the embedding for troublesome.
troubling is at index 15554
Saved the embedding for troubling.
trusting is at index 28969
Saved the embedding for trusting.
trustworthy is at index 32101
Saved the embedding for trustworthy.
tumultuous is at index 23787
Saved the embedding for tumultuous.
turbulent is at

Saved the embedding for violence.
violent is at index 4153
Saved the embedding for violent.
viperous is at index 748
Saved the embedding for viperous.
vituperative is at index 14306
Saved the embedding for vituperative.
vocal is at index 7578
Saved the embedding for vocal.
vocalized is at index 7578
Saved the embedding for vocalized.
vulgar is at index 28792
Saved the embedding for vulgar.
vulnerability is at index 15661
Saved the embedding for vulnerability.
vulnerable is at index 4478
Saved the embedding for vulnerable.
wacky is at index 885
Saved the embedding for wacky.
waiting is at index 2445
Saved the embedding for waiting.
wanted is at index 770
Saved the embedding for wanted.
wanting is at index 6923
Saved the embedding for wanting.
wanton is at index 236
Saved the embedding for wanton.
wariness is at index 997
Saved the embedding for wariness.
warm is at index 3279
Saved the embedding for warm.
wary is at index 13441
Saved the embedding for wary.
wasted is at index 14260
Save